# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_c
　　セッション1のAさんのスケーリングデータ　→　data1_c_label_50

In [2]:
%time
#sensor_dataの読み込み

# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

data1_c_label_50 = pd.read_csv('emotion_label/session1/C/emotion_C_20201111_1_50.csv')
data1_c_label_60 = pd.read_csv('emotion_label/session1/C/emotion_C_20201111_1_60.csv')
data1_c_label_70 = pd.read_csv('emotion_label/session1/C/emotion_C_20201111_1_70.csv')
data1_c_label_80 = pd.read_csv('emotion_label/session1/C/emotion_C_20201111_1_80.csv')

# data2_c_label_50 = pd.read_csv('emotion_label/C/emotion_C_20201111_2_50.csv')
# data2_c_label_60 = pd.read_csv('emotion_label/C/emotion_C_20201111_2_60.csv')
# data2_c_label_70 = pd.read_csv('emotion_label/C/emotion_C_20201111_2_70.csv')
# data2_c_label_80 = pd.read_csv('emotion_label/C/emotion_C_20201111_2_80.csv')

Wall time: 0 ns


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data1_c_label_50

,start_time,finish_time,emotion_level
0,5:37,5:41,3
1,5:38,5:42,3
2,5:39,5:43,3
3,5:40,5:44,3
4,5:41,5:45,3
...,...,...,...
577,20:19,20:23,1
578,20:20,20:24,1
579,20:21,20:25,1
580,20:22,20:26,1


In [4]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

data1_c_label_50 = dropna(data1_c_label_50)
data1_c_label_60 = dropna(data1_c_label_60)
data1_c_label_70 = dropna(data1_c_label_70)
data1_c_label_80 = dropna(data1_c_label_80)

# data2_c_label_50 = dropna(data2_c_label_50)
# data2_c_label_60 = dropna(data2_c_label_60)
# data2_c_label_70 = dropna(data2_c_label_70)
# data2_c_label_80 = dropna(data2_c_label_80)

In [5]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=0)
    return a

data1_c_label_50 = dropna(data1_c_label_50)
data1_c_label_60 = dropna(data1_c_label_60)
data1_c_label_70 = dropna(data1_c_label_70)
data1_c_label_80 = dropna(data1_c_label_80)

# data2_c_label_50 = dropna(data2_c_label_50)
# data2_c_label_60 = dropna(data2_c_label_60)
# data2_c_label_70 = dropna(data2_c_label_70)
# data2_c_label_80 = dropna(data2_c_label_80)

In [6]:
data1_c_label_50

,start_time,finish_time,emotion_level
0,5:37,5:41,3
1,5:38,5:42,3
2,5:39,5:43,3
3,5:40,5:44,3
4,5:41,5:45,3
...,...,...,...
577,20:19,20:23,1
578,20:20,20:24,1
579,20:21,20:25,1
580,20:22,20:26,1


In [7]:
#生データの確認
display(data1_c)
display(data1_c_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,12:19:53.940,10149,-29,1155,-1160,-761,-301
1,ags,12:19:53.945,10166,-34,1118,-1257,-761,-350
2,ags,12:19:53.950,10242,20,1084,-1337,-791,-375
3,ags,12:19:53.955,10181,-27,1050,-1416,-755,-368
4,ags,12:19:53.960,10244,-61,1089,-1550,-730,-368
...,...,...,...,...,...,...,...,...
460873,ags,12:58:19.965,3821,2161,8530,-203,-377,-113
460874,ags,12:58:19.970,4175,2007,8545,-257,-96,-387
460875,ags,12:58:19.975,4338,1946,8491,-325,330,-685
460876,ags,12:58:19.980,4167,1966,8533,-379,616,-954


,start_time,finish_time,emotion_level
0,5:37,5:41,3
1,5:38,5:42,3
2,5:39,5:43,3
3,5:40,5:44,3
4,5:41,5:45,3
...,...,...,...
577,20:19,20:23,1
578,20:20,20:24,1
579,20:21,20:25,1
580,20:22,20:26,1


In [8]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data1_c)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
# norm_order(data2_c)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [9]:
data1_c

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,12:19:53.940,10149,-29,1155,-1160,-761,-301,10214.551728,1419.620372
1,ags,12:19:53.945,10166,-34,1118,-1257,-761,-350,10227.347457,1510.519778
2,ags,12:19:53.950,10242,20,1084,-1337,-791,-375,10299.224243,1598.084791
3,ags,12:19:53.955,10181,-27,1050,-1416,-755,-368,10235.037372,1646.361139
4,ags,12:19:53.960,10244,-61,1089,-1550,-730,-368,10301.901669,1752.376672
...,...,...,...,...,...,...,...,...,...,...
460873,ags,12:58:19.965,3821,2161,8530,-203,-377,-113,9593.271705,442.839700
460874,ags,12:58:19.970,4175,2007,8545,-257,-96,-387,9719.861059,474.377487
460875,ags,12:58:19.975,4338,1946,8491,-325,330,-685,9731.507643,826.891770
460876,ags,12:58:19.980,4167,1966,8533,-379,616,-954,9697.480807,1197.168743


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [10]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data1_c)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

# datetime_order(data2_c)
# datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [11]:
data1_c

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:19:53.940,10149,-29,1155,-1160,-761,-301,10214.551728,1419.620372
1,ags,2020-11-11 12:19:53.945,10166,-34,1118,-1257,-761,-350,10227.347457,1510.519778
2,ags,2020-11-11 12:19:53.950,10242,20,1084,-1337,-791,-375,10299.224243,1598.084791
3,ags,2020-11-11 12:19:53.955,10181,-27,1050,-1416,-755,-368,10235.037372,1646.361139
4,ags,2020-11-11 12:19:53.960,10244,-61,1089,-1550,-730,-368,10301.901669,1752.376672
...,...,...,...,...,...,...,...,...,...,...
460873,ags,2020-11-11 12:58:19.965,3821,2161,8530,-203,-377,-113,9593.271705,442.839700
460874,ags,2020-11-11 12:58:19.970,4175,2007,8545,-257,-96,-387,9719.861059,474.377487
460875,ags,2020-11-11 12:58:19.975,4338,1946,8491,-325,330,-685,9731.507643,826.891770
460876,ags,2020-11-11 12:58:19.980,4167,1966,8533,-379,616,-954,9697.480807,1197.168743


In [12]:
data1_c_label_50

,start_time,finish_time,emotion_level
0,5:37,5:41,3
1,5:38,5:42,3
2,5:39,5:43,3
3,5:40,5:44,3
4,5:41,5:45,3
...,...,...,...
577,20:19,20:23,1
578,20:20,20:24,1
579,20:21,20:25,1
580,20:22,20:26,1


In [13]:
data1_c_label_50['start_time'] = pd.to_datetime(data1_c_label_50['start_time'], format='%M:%S')
data1_c_label_50['finish_time'] = pd.to_datetime(data1_c_label_50['finish_time'], format='%M:%S')

In [14]:
data1_c_label_60['start_time'] = pd.to_datetime(data1_c_label_60['start_time'], format='%M:%S')
data1_c_label_60['finish_time'] = pd.to_datetime(data1_c_label_60['finish_time'], format='%M:%S')

In [15]:
data1_c_label_70['start_time'] = pd.to_datetime(data1_c_label_70['start_time'], format='%M:%S')
data1_c_label_70['finish_time'] = pd.to_datetime(data1_c_label_70['finish_time'], format='%M:%S')

In [16]:
data1_c_label_80['start_time'] = pd.to_datetime(data1_c_label_80['start_time'], format='%M:%S')
data1_c_label_80['finish_time'] = pd.to_datetime(data1_c_label_80['finish_time'], format='%M:%S')

In [17]:
# data2_c_label_50['start_time'] = pd.to_datetime(data2_c_label_50['start_time'], format='%M:%S')
# data2_c_label_50['finish_time'] = pd.to_datetime(data2_c_label_50['finish_time'], format='%M:%S')

In [18]:
# data2_c_label_60['start_time'] = pd.to_datetime(data2_c_label_60['start_time'], format='%M:%S')
# data2_c_label_60['finish_time'] = pd.to_datetime(data2_c_label_60['finish_time'], format='%M:%S')

In [19]:
# data2_c_label_70['start_time'] = pd.to_datetime(data2_c_label_70['start_time'], format='%M:%S')
# data2_c_label_70['finish_time'] = pd.to_datetime(data2_c_label_70['finish_time'], format='%M:%S')

In [20]:
# data2_c_label_80['start_time'] = pd.to_datetime(data2_c_label_80['start_time'], format='%M:%S')
# data2_c_label_80['finish_time'] = pd.to_datetime(data2_c_label_80['finish_time'], format='%M:%S')

In [21]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_c)
display(data1_c_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:19:53.940,10149,-29,1155,-1160,-761,-301,10214.551728,1419.620372
1,ags,2020-11-11 12:19:53.945,10166,-34,1118,-1257,-761,-350,10227.347457,1510.519778
2,ags,2020-11-11 12:19:53.950,10242,20,1084,-1337,-791,-375,10299.224243,1598.084791
3,ags,2020-11-11 12:19:53.955,10181,-27,1050,-1416,-755,-368,10235.037372,1646.361139
4,ags,2020-11-11 12:19:53.960,10244,-61,1089,-1550,-730,-368,10301.901669,1752.376672
...,...,...,...,...,...,...,...,...,...,...
460873,ags,2020-11-11 12:58:19.965,3821,2161,8530,-203,-377,-113,9593.271705,442.839700
460874,ags,2020-11-11 12:58:19.970,4175,2007,8545,-257,-96,-387,9719.861059,474.377487
460875,ags,2020-11-11 12:58:19.975,4338,1946,8491,-325,330,-685,9731.507643,826.891770
460876,ags,2020-11-11 12:58:19.980,4167,1966,8533,-379,616,-954,9697.480807,1197.168743


,start_time,finish_time,emotion_level
0,1900-01-01 00:05:37,1900-01-01 00:05:41,3
1,1900-01-01 00:05:38,1900-01-01 00:05:42,3
2,1900-01-01 00:05:39,1900-01-01 00:05:43,3
3,1900-01-01 00:05:40,1900-01-01 00:05:44,3
4,1900-01-01 00:05:41,1900-01-01 00:05:45,3
...,...,...,...
577,1900-01-01 00:20:19,1900-01-01 00:20:23,1
578,1900-01-01 00:20:20,1900-01-01 00:20:24,1
579,1900-01-01 00:20:21,1900-01-01 00:20:25,1
580,1900-01-01 00:20:22,1900-01-01 00:20:26,1


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [22]:
data1_c_label_50['start_time'] = data1_c_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label_50['finish_time'] = data1_c_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [23]:
data1_c_label_60['start_time'] = data1_c_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label_60['finish_time'] = data1_c_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [24]:
data1_c_label_70['start_time'] = data1_c_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label_70['finish_time'] = data1_c_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [25]:
data1_c_label_80['start_time'] = data1_c_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label_80['finish_time'] = data1_c_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [26]:
# data2_c_label_50['start_time'] = data2_c_label_50['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_c_label_50['finish_time'] = data2_c_label_50['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [27]:
# data2_c_label_60['start_time'] = data2_c_label_60['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_c_label_60['finish_time'] = data2_c_label_60['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [28]:
# data2_c_label_70['start_time'] = data2_c_label_70['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_c_label_70['finish_time'] = data2_c_label_70['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [29]:
# data2_c_label_80['start_time'] = data2_c_label_80['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
# data2_c_label_80['finish_time'] = data2_c_label_80['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [30]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_c_label_80)

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:07,2020-11-11 12:40:11,3
1,2020-11-11 12:40:08,2020-11-11 12:40:12,3
2,2020-11-11 12:40:09,2020-11-11 12:40:13,3
3,2020-11-11 12:40:10,2020-11-11 12:40:14,3
4,2020-11-11 12:40:11,2020-11-11 12:40:15,3
...,...,...,...
499,2020-11-11 12:54:49,2020-11-11 12:54:53,1
500,2020-11-11 12:54:50,2020-11-11 12:54:54,1
501,2020-11-11 12:54:51,2020-11-11 12:54:55,1
502,2020-11-11 12:54:52,2020-11-11 12:54:56,1


In [31]:
#datetime化したラベルデータを保存
# data1_c_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_c_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_c_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_c_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_c_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_c_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_c_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_c_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [32]:
#datetime化した加速度及び角速度データを保存
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [33]:
#時刻をインデックスに
data1_c.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

# data2_c.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [34]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_c)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 12:19:53.940,ags,10149,-29,1155,-1160,-761,-301,10214.551728,1419.620372
2020-11-11 12:19:53.945,ags,10166,-34,1118,-1257,-761,-350,10227.347457,1510.519778
2020-11-11 12:19:53.950,ags,10242,20,1084,-1337,-791,-375,10299.224243,1598.084791
2020-11-11 12:19:53.955,ags,10181,-27,1050,-1416,-755,-368,10235.037372,1646.361139
2020-11-11 12:19:53.960,ags,10244,-61,1089,-1550,-730,-368,10301.901669,1752.376672
...,...,...,...,...,...,...,...,...,...
2020-11-11 12:58:19.965,ags,3821,2161,8530,-203,-377,-113,9593.271705,442.839700
2020-11-11 12:58:19.970,ags,4175,2007,8545,-257,-96,-387,9719.861059,474.377487
2020-11-11 12:58:19.975,ags,4338,1946,8491,-325,330,-685,9731.507643,826.891770


In [35]:
data1_c_label_50["start_time"] = data1_c_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label_50["finish_time"] = data1_c_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [36]:
data1_c_label_60["start_time"] = data1_c_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label_60["finish_time"] = data1_c_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [37]:
data1_c_label_70["start_time"] = data1_c_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label_70["finish_time"] = data1_c_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
data1_c_label_80["start_time"] = data1_c_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label_80["finish_time"] = data1_c_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
# data2_c_label_50["start_time"] = data2_c_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_50["finish_time"] = data2_c_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [40]:
# data2_c_label_60["start_time"] = data2_c_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_60["finish_time"] = data2_c_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [41]:
# data2_c_label_70["start_time"] = data2_c_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_70["finish_time"] = data2_c_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [42]:
# data2_c_label_80["start_time"] = data2_c_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_80["finish_time"] = data2_c_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [43]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_c_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:07,2020-11-11 12:40:11,3
1,2020-11-11 12:40:08,2020-11-11 12:40:12,3
2,2020-11-11 12:40:09,2020-11-11 12:40:13,3
3,2020-11-11 12:40:10,2020-11-11 12:40:14,3
4,2020-11-11 12:40:11,2020-11-11 12:40:15,3
...,...,...,...
528,2020-11-11 12:54:49,2020-11-11 12:54:53,1
529,2020-11-11 12:54:50,2020-11-11 12:54:54,1
530,2020-11-11 12:54:51,2020-11-11 12:54:55,1
531,2020-11-11 12:54:52,2020-11-11 12:54:56,1


In [44]:
#startとfinishをリスト化
data1_c_50_start_time = list(data1_c_label_50["start_time"])
data1_c_50_finish_time = list(data1_c_label_50["finish_time"])

data1_c_60_start_time = list(data1_c_label_60["start_time"])
data1_c_60_finish_time = list(data1_c_label_60["finish_time"])

data1_c_70_start_time = list(data1_c_label_70["start_time"])
data1_c_70_finish_time = list(data1_c_label_70["finish_time"])

data1_c_80_start_time = list(data1_c_label_80["start_time"])
data1_c_80_finish_time = list(data1_c_label_80["finish_time"])

# data2_c_50_start_time = list(data2_c_label_50["start_time"])
# data2_c_50_finish_time = list(data2_c_label_50["finish_time"])

# data2_c_60_start_time = list(data2_c_label_60["start_time"])
# data2_c_60_finish_time = list(data2_c_label_60["finish_time"])

# data2_c_70_start_time = list(data2_c_label_70["start_time"])
# data2_c_70_finish_time = list(data2_c_label_70["finish_time"])

# data2_c_80_start_time = list(data2_c_label_80["start_time"])
# data2_c_80_finish_time = list(data2_c_label_80["finish_time"])

In [45]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

data1_c_record_1 = sensor_segmentation_order(data1_c, data1_c_50_start_time, data1_c_50_finish_time, data1_c_label_50)
data1_c_record_2 = sensor_segmentation_order(data1_c, data1_c_60_start_time, data1_c_60_finish_time, data1_c_label_60)
data1_c_record_3 = sensor_segmentation_order(data1_c, data1_c_70_start_time, data1_c_70_finish_time, data1_c_label_70)
data1_c_record_4 = sensor_segmentation_order(data1_c, data1_c_80_start_time, data1_c_80_finish_time, data1_c_label_80)

# data2_c_record_1 = sensor_segmentation_order(data2_c, data2_c_50_start_time, data2_c_50_finish_time, data2_c_label_50)
# data2_c_record_2 = sensor_segmentation_order(data2_c, data2_c_60_start_time, data2_c_60_finish_time, data2_c_label_60)
# data2_c_record_3 = sensor_segmentation_order(data2_c, data2_c_70_start_time, data2_c_70_finish_time, data2_c_label_70)
# data2_c_record_4 = sensor_segmentation_order(data2_c, data2_c_80_start_time, data2_c_80_finish_time, data2_c_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [46]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

data1_c_50_record_mean = acc_gyr_mean_order(data1_c_record_1, data1_c_label_50)
data1_c_60_record_mean = acc_gyr_mean_order(data1_c_record_2, data1_c_label_60)
data1_c_70_record_mean = acc_gyr_mean_order(data1_c_record_3, data1_c_label_70)
data1_c_80_record_mean = acc_gyr_mean_order(data1_c_record_4, data1_c_label_80)

# data2_c_50_record_mean = acc_gyr_mean_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_mean = acc_gyr_mean_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_mean = acc_gyr_mean_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_mean = acc_gyr_mean_order(data2_c_record_4, data2_c_label_80)

# a = data1_c_50_record_mean
# b = data1_c_60_record_mean
# c = data1_c_70_record_mean
# d = data1_c_80_record_mean

# e = data2_c_50_record_mean
# f = data2_c_60_record_mean
# g = data2_c_70_record_mean
# h = data2_c_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [47]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

data1_c_50_record_var = acc_gyr_var_order(data1_c_record_1, data1_c_label_50)
data1_c_60_record_var = acc_gyr_var_order(data1_c_record_2, data1_c_label_60)
data1_c_70_record_var = acc_gyr_var_order(data1_c_record_3, data1_c_label_70)
data1_c_80_record_var = acc_gyr_var_order(data1_c_record_4, data1_c_label_80)

# data2_c_50_record_var = acc_gyr_var_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_var = acc_gyr_var_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_var = acc_gyr_var_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_var = acc_gyr_var_order(data2_c_record_4, data2_c_label_80)

# a = data1_c_50_record_var
# b = data1_c_60_record_var
# c = data1_c_70_record_var
# d = data1_c_80_record_var

# e = data2_c_50_record_var
# f = data2_c_60_record_var
# g = data2_c_70_record_var
# h = data2_c_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [48]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

data1_c_50_record_max = acc_gyr_max_order(data1_c_record_1, data1_c_label_50)
data1_c_60_record_max = acc_gyr_max_order(data1_c_record_2, data1_c_label_60)
data1_c_70_record_max = acc_gyr_max_order(data1_c_record_3, data1_c_label_70)
data1_c_80_record_max = acc_gyr_max_order(data1_c_record_4, data1_c_label_80)

# data2_c_50_record_max = acc_gyr_max_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_max = acc_gyr_max_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_max = acc_gyr_max_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_max = acc_gyr_max_order(data2_c_record_4, data2_c_label_80)

# a = data1_c_50_record_max
# b = data1_c_60_record_max
# c = data1_c_70_record_max
# d = data1_c_80_record_max

# e = data2_c_50_record_max
# f = data2_c_60_record_max
# g = data2_c_70_record_max
# h = data2_c_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [49]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

data1_c_50_record_min = acc_gyr_min_order(data1_c_record_1, data1_c_label_50)
data1_c_60_record_min = acc_gyr_min_order(data1_c_record_2, data1_c_label_60)
data1_c_70_record_min = acc_gyr_min_order(data1_c_record_3, data1_c_label_70)
data1_c_80_record_min = acc_gyr_min_order(data1_c_record_4, data1_c_label_80)

# data2_c_50_record_min = acc_gyr_min_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_min = acc_gyr_min_order(data2_b_record_2, data2_c_label_60)
# data2_c_70_record_min = acc_gyr_min_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_min = acc_gyr_min_order(data2_d_record_4, data2_c_label_80)

# a = data1_c_50_record_min
# b = data1_c_60_record_min
# c = data1_c_70_record_min
# d = data1_c_80_record_min

# e = data2_c_50_record_min
# f = data2_c_60_record_min
# g = data2_c_70_record_min
# h = data2_c_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [50]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

data1_c_50_record_fq_amp_max = fq_amp_max_order(data1_c_record_1, data1_c_label_50)
data1_c_60_record_fq_amp_max = fq_amp_max_order(data1_c_record_2, data1_c_label_60)
data1_c_70_record_fq_amp_max = fq_amp_max_order(data1_c_record_3, data1_c_label_70)
data1_c_80_record_fq_amp_max = fq_amp_max_order(data1_c_record_4, data1_c_label_80)

# data2_c_50_record_fq_amp_max = fq_amp_max_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_fq_amp_max = fq_amp_max_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_fq_amp_max = fq_amp_max_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_fq_amp_max = fq_amp_max_order(data2_c_record_4, data2_c_label_80)

# a = data1_c_50_record_fq_amp_max
# b = data1_c_60_record_fq_amp_max
# c = data1_c_70_record_fq_amp_max
# d = data1_c_80_record_fq_amp_max

# e = data2_c_50_record_fq_amp_max
# f = data2_c_60_record_fq_amp_max
# g = data2_c_70_record_fq_amp_max
# h = data2_c_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [51]:
#全特徴量の行列の導出
data1_c_50_record_all_feature = pd.concat([data1_c_50_record_mean, data1_c_50_record_var, data1_c_50_record_max, data1_c_50_record_min, data1_c_50_record_fq_amp_max], axis=1, ignore_index=False)
data1_c_60_record_all_feature = pd.concat([data1_c_60_record_mean, data1_c_60_record_var, data1_c_60_record_max, data1_c_60_record_min, data1_c_60_record_fq_amp_max], axis=1, ignore_index=False)
data1_c_70_record_all_feature = pd.concat([data1_c_70_record_mean, data1_c_70_record_var, data1_c_70_record_max, data1_c_70_record_min, data1_c_70_record_fq_amp_max], axis=1, ignore_index=False)
data1_c_80_record_all_feature = pd.concat([data1_c_80_record_mean, data1_c_80_record_var, data1_c_80_record_max, data1_c_80_record_min, data1_c_80_record_fq_amp_max], axis=1, ignore_index=False)

# data2_c_50_record_all_feature = pd.concat([data2_c_50_record_mean, data2_c_50_record_var, data2_c_50_record_max, data2_c_50_record_min, data2_c_50_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_60_record_all_feature = pd.concat([data2_c_60_record_mean, data2_c_60_record_var, data2_c_60_record_max, data2_c_60_record_min, data2_c_60_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_70_record_all_feature = pd.concat([data2_c_70_record_mean, data2_c_70_record_var, data2_c_70_record_max, data2_c_70_record_min, data2_c_70_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_80_record_all_feature = pd.concat([data2_c_80_record_mean, data2_c_80_record_var, data2_c_80_record_max, data2_c_80_record_min, data2_c_80_record_fq_amp_max], axis=1, ignore_index=False)

In [52]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_c_60_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,10145.054,-128.325,-49.914,92.539,-54.404,-163.588,101488.985084,94748.747375,265112.208604,500053.536479,...,8892,-1151,-1352,-1721,-1572,-1100,7.045010,215.780697,0.782779,383.845702
1,10141.041,-71.737,-14.066,119.790,143.093,-101.599,96020.519319,82529.385831,270087.955644,363144.329900,...,8892,-1151,-1352,-1715,-1572,-1100,7.045010,282.415181,0.391389,408.414881
2,10137.723,-121.398,55.933,67.763,20.992,-112.556,74876.710271,69993.853596,229295.390511,264111.792831,...,8892,-1114,-1220,-1715,-1584,-1100,7.827789,198.499846,0.391389,283.779818
3,10144.075,-159.622,-124.623,47.819,-69.766,-68.971,39539.789375,53107.783116,225850.628871,154007.660239,...,9409,-665,-1220,-1245,-1676,-692,5.870841,123.429122,0.782779,138.589461
4,10138.438,-190.679,-263.798,81.040,-5.743,-119.497,25226.800156,38448.885959,213287.207196,108802.432400,...,9409,-638,-1220,-1190,-1676,-692,6.653620,87.220321,0.782779,204.261997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,9355.494,-322.234,-3774.870,62.091,-51.138,-42.745,9251.137964,6333.579244,33893.809100,6592.376719,...,8904,-868,-4621,-410,-1749,-234,7.045010,53.968046,1.174168,92.264464
557,9247.038,-480.127,-3976.078,105.334,-86.059,-36.297,68856.728556,96283.466871,234185.647916,34141.516444,...,8191,-1456,-5580,-410,-1749,-393,5.870841,60.882606,1.174168,91.117451
558,9179.877,-614.520,-4107.153,86.109,-19.162,-49.036,77069.761871,120633.571600,270457.833591,39076.329119,...,8191,-1456,-5580,-410,-1749,-393,5.870841,67.701913,0.391389,79.096249
559,9133.082,-709.777,-4202.876,91.537,-36.808,-81.210,65760.795276,100313.837271,234224.912624,39086.052631,...,8191,-1456,-5580,-410,-1749,-442,12.133072,41.725930,0.391389,514.251137


In [53]:
#各特徴量データの保存
data1_c_50_record_all_feature.to_csv('feature/C/data1_c_50_record_all_feature.csv', index=False)
data1_c_60_record_all_feature.to_csv('feature/C/data1_c_60_record_all_feature.csv', index=False)
data1_c_70_record_all_feature.to_csv('feature/C/data1_c_70_record_all_feature.csv', index=False)
data1_c_80_record_all_feature.to_csv('feature/C/data1_c_80_record_all_feature.csv', index=False)

# data2_c_50_record_all_feature.to_csv('feature/C/data2_c_50_record_all_feature.csv', index=False)
# data2_c_60_record_all_feature.to_csv('feature/C/data2_c_60_record_all_feature.csv', index=False)
# data2_c_70_record_all_feature.to_csv('feature/C/data2_c_70_record_all_feature.csv', index=False)
# data2_c_80_record_all_feature.to_csv('feature/C/data2_c_80_record_all_feature.csv', index=False)

In [54]:
data1_c_label_50

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:07,2020-11-11 12:40:11,3
1,2020-11-11 12:40:08,2020-11-11 12:40:12,3
2,2020-11-11 12:40:09,2020-11-11 12:40:13,3
3,2020-11-11 12:40:10,2020-11-11 12:40:14,3
4,2020-11-11 12:40:11,2020-11-11 12:40:15,3
...,...,...,...
577,2020-11-11 12:54:49,2020-11-11 12:54:53,1
578,2020-11-11 12:54:50,2020-11-11 12:54:54,1
579,2020-11-11 12:54:51,2020-11-11 12:54:55,1
580,2020-11-11 12:54:52,2020-11-11 12:54:56,1


In [55]:
data1_c_label_50_time = data1_c_label_50.drop("emotion_level", axis=1)
data1_c_label_60_time = data1_c_label_60.drop("emotion_level", axis=1)
data1_c_label_70_time = data1_c_label_70.drop("emotion_level", axis=1)
data1_c_label_80_time = data1_c_label_80.drop("emotion_level", axis=1)

# data2_c_label_50_time = data2_c_label_50.drop("emotion_level", axis=1)
# data2_c_label_60_time = data2_c_label_60.drop("emotion_level", axis=1)
# data2_c_label_70_time = data2_c_label_70.drop("emotion_level", axis=1)
# data2_c_label_80_time = data2_c_label_80.drop("emotion_level", axis=1)

In [56]:
data1_c_label_50_time

,start_time,finish_time
0,2020-11-11 12:40:07,2020-11-11 12:40:11
1,2020-11-11 12:40:08,2020-11-11 12:40:12
2,2020-11-11 12:40:09,2020-11-11 12:40:13
3,2020-11-11 12:40:10,2020-11-11 12:40:14
4,2020-11-11 12:40:11,2020-11-11 12:40:15
...,...,...
577,2020-11-11 12:54:49,2020-11-11 12:54:53
578,2020-11-11 12:54:50,2020-11-11 12:54:54
579,2020-11-11 12:54:51,2020-11-11 12:54:55
580,2020-11-11 12:54:52,2020-11-11 12:54:56


In [57]:
data1_c_50_record_all_feature = pd.concat([data1_c_label_50_time, data1_c_50_record_all_feature], axis=1)
data1_c_60_record_all_feature = pd.concat([data1_c_label_60_time, data1_c_60_record_all_feature], axis=1)
data1_c_70_record_all_feature = pd.concat([data1_c_label_70_time, data1_c_70_record_all_feature], axis=1)
data1_c_80_record_all_feature = pd.concat([data1_c_label_80_time, data1_c_80_record_all_feature], axis=1)

# data2_c_50_record_all_feature = pd.concat([data2_c_label_50_time, data2_c_50_record_all_feature], axis=1)
# data2_c_60_record_all_feature = pd.concat([data2_c_label_60_time, data2_c_60_record_all_feature], axis=1)
# data2_c_70_record_all_feature = pd.concat([data2_c_label_70_time, data2_c_70_record_all_feature], axis=1)
# data2_c_80_record_all_feature = pd.concat([data2_c_label_80_time, data2_c_80_record_all_feature], axis=1)

In [58]:
data1_c_60_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:40:07,2020-11-11 12:40:11,10145.054,-128.325,-49.914,92.539,-54.404,-163.588,101488.985084,94748.747375,...,8892,-1151,-1352,-1721,-1572,-1100,7.045010,215.780697,0.782779,383.845702
1,2020-11-11 12:40:08,2020-11-11 12:40:12,10141.041,-71.737,-14.066,119.790,143.093,-101.599,96020.519319,82529.385831,...,8892,-1151,-1352,-1715,-1572,-1100,7.045010,282.415181,0.391389,408.414881
2,2020-11-11 12:40:09,2020-11-11 12:40:13,10137.723,-121.398,55.933,67.763,20.992,-112.556,74876.710271,69993.853596,...,8892,-1114,-1220,-1715,-1584,-1100,7.827789,198.499846,0.391389,283.779818
3,2020-11-11 12:40:10,2020-11-11 12:40:14,10144.075,-159.622,-124.623,47.819,-69.766,-68.971,39539.789375,53107.783116,...,9409,-665,-1220,-1245,-1676,-692,5.870841,123.429122,0.782779,138.589461
4,2020-11-11 12:40:11,2020-11-11 12:40:15,10138.438,-190.679,-263.798,81.040,-5.743,-119.497,25226.800156,38448.885959,...,9409,-638,-1220,-1190,-1676,-692,6.653620,87.220321,0.782779,204.261997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,2020-11-11 12:54:49,2020-11-11 12:54:53,9355.494,-322.234,-3774.870,62.091,-51.138,-42.745,9251.137964,6333.579244,...,8904,-868,-4621,-410,-1749,-234,7.045010,53.968046,1.174168,92.264464
557,2020-11-11 12:54:50,2020-11-11 12:54:54,9247.038,-480.127,-3976.078,105.334,-86.059,-36.297,68856.728556,96283.466871,...,8191,-1456,-5580,-410,-1749,-393,5.870841,60.882606,1.174168,91.117451
558,2020-11-11 12:54:51,2020-11-11 12:54:55,9179.877,-614.520,-4107.153,86.109,-19.162,-49.036,77069.761871,120633.571600,...,8191,-1456,-5580,-410,-1749,-393,5.870841,67.701913,0.391389,79.096249
559,2020-11-11 12:54:52,2020-11-11 12:54:56,9133.082,-709.777,-4202.876,91.537,-36.808,-81.210,65760.795276,100313.837271,...,8191,-1456,-5580,-410,-1749,-442,12.133072,41.725930,0.391389,514.251137


In [59]:
data1_c_50_record_all_feature.to_csv('feature/C/data1_c_50_record_all_feature.csv', index=False)
data1_c_60_record_all_feature.to_csv('feature/C/data1_c_60_record_all_feature.csv', index=False)
data1_c_70_record_all_feature.to_csv('feature/C/data1_c_70_record_all_feature.csv', index=False)
data1_c_80_record_all_feature.to_csv('feature/C/data1_c_80_record_all_feature.csv', index=False)

# data2_c_50_record_all_feature.to_csv('feature/C/data2_c_50_record_all_feature.csv', index=False)
# data2_c_60_record_all_feature.to_csv('feature/C/data2_c_60_record_all_feature.csv', index=False)
# data2_c_70_record_all_feature.to_csv('feature/C/data2_c_70_record_all_feature.csv', index=False)
# data2_c_80_record_all_feature.to_csv('feature/C/data2_c_80_record_all_feature.csv', index=False)

In [60]:
data1_c_60_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:40:07,2020-11-11 12:40:11,10145.054,-128.325,-49.914,92.539,-54.404,-163.588,101488.985084,94748.747375,...,8892,-1151,-1352,-1721,-1572,-1100,7.045010,215.780697,0.782779,383.845702
1,2020-11-11 12:40:08,2020-11-11 12:40:12,10141.041,-71.737,-14.066,119.790,143.093,-101.599,96020.519319,82529.385831,...,8892,-1151,-1352,-1715,-1572,-1100,7.045010,282.415181,0.391389,408.414881
2,2020-11-11 12:40:09,2020-11-11 12:40:13,10137.723,-121.398,55.933,67.763,20.992,-112.556,74876.710271,69993.853596,...,8892,-1114,-1220,-1715,-1584,-1100,7.827789,198.499846,0.391389,283.779818
3,2020-11-11 12:40:10,2020-11-11 12:40:14,10144.075,-159.622,-124.623,47.819,-69.766,-68.971,39539.789375,53107.783116,...,9409,-665,-1220,-1245,-1676,-692,5.870841,123.429122,0.782779,138.589461
4,2020-11-11 12:40:11,2020-11-11 12:40:15,10138.438,-190.679,-263.798,81.040,-5.743,-119.497,25226.800156,38448.885959,...,9409,-638,-1220,-1190,-1676,-692,6.653620,87.220321,0.782779,204.261997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,2020-11-11 12:54:49,2020-11-11 12:54:53,9355.494,-322.234,-3774.870,62.091,-51.138,-42.745,9251.137964,6333.579244,...,8904,-868,-4621,-410,-1749,-234,7.045010,53.968046,1.174168,92.264464
557,2020-11-11 12:54:50,2020-11-11 12:54:54,9247.038,-480.127,-3976.078,105.334,-86.059,-36.297,68856.728556,96283.466871,...,8191,-1456,-5580,-410,-1749,-393,5.870841,60.882606,1.174168,91.117451
558,2020-11-11 12:54:51,2020-11-11 12:54:55,9179.877,-614.520,-4107.153,86.109,-19.162,-49.036,77069.761871,120633.571600,...,8191,-1456,-5580,-410,-1749,-393,5.870841,67.701913,0.391389,79.096249
559,2020-11-11 12:54:52,2020-11-11 12:54:56,9133.082,-709.777,-4202.876,91.537,-36.808,-81.210,65760.795276,100313.837271,...,8191,-1456,-5580,-410,-1749,-442,12.133072,41.725930,0.391389,514.251137
